In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np

Type in the course and date for the video below, using the following format as an example:

- course = Psych100
- theDate = yyyy-mm-dd

Course and date should be exactly as they appear in the video title

In [ ]:
# Replace these fields
course = 'Psych261'
theDate = '2019-11-04'

In [ ]:
grabs = [g for g in os.listdir('./../output/{}'.format(course)) if g.split('_')[0] == theDate]
grabs = [g for g in grabs if g.split('.')[-1] == 'gif']
grabs = sorted(grabs, key=lambda x: float(x.split('_')[-1].split('.gif')[0]))
print('{} frames found'.format(len(grabs)))
onsets = [int(g.split('.gif')[0].split('_')[-1]) for g in grabs]
fig, ax = plt.subplots(figsize = (25,2))
plt.scatter(onsets, np.ones(len(onsets)))
plt.show()

In [ ]:
durs = [100]
starts = []
stops = []
vid = False
for i, (ons, im) in enumerate(zip(onsets, grabs)):
    if i > 1:
        dur = ons - prev
        durs.append(dur)
    if len(durs) > 10:
        avgdur = np.mean(durs[(i-10):i])
        if avgdur < 201:
            if not vid:
                print('{} likely a video onset'.format(grabs[i-11]))
                vid = True
                starts.append(i-11)
        elif vid:
            print('{} likely a video offset'.format(grabs[i-1]))
            vid = False
            stops.append(i-1)
    prev = ons*1
if len(starts) > 0:
    videodoc = np.vstack((starts, stops)).T
    np.save('./../output/{}/{}_videos.npy'.format(course, theDate), videodoc)



In [ ]:
images = []
vidnum = 0
doc = True
for i, im in enumerate(grabs):
    if i == 0:
        im0 = Image.open('./../output/{}/{}'.format(course, im))
    else:
        if doc:
            im1 = Image.open('./../output/{}/{}'.format(course, im))
            if len(starts) > vidnum and i-1 == starts[vidnum]:
                nextstop = stops[vidnum]
                vidnum +=1
                images.append(Image.fromarray(np.array(im1)-np.array(im1)))
                doc = False
            else:
                images.append(im1)
        else:
            if i+1 == nextstop:
                doc = True

pdfname = "./../output/{}/{}.pdf".format(course, theDate)
subname = "./../output/{}/{}_subset.pdf".format(course, theDate)

im0.save(pdfname, "PDF", resolution=100.0, save_all=True, append_images=images)
print('saved PDF for {} on {}'.format(course, theDate))